# Data cleansing

This is a jupyter notebook for data cleansing of domains got from the crawler "Reveal True Domains", which has returned true domains after redirecting from Heureka.cz

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('true_domains.csv', names=['name','heureka', 'true_domain'])

## Split data to actual domains and errors
Actual true domains always starts with http and errors starts with returned status code, message of "Connection refused" or message "Got Expection named 'Exception ClassName'".

In [5]:
tdf=df[df['true_domain'].str.startswith('http')]
errors=df[~df['true_domain'].str.startswith('http')]

## Errors validation

Far most of the redirection were refused at the connection. The main issue here could be that Heureka.cz was able to detect our crawler and refused to redirect our request. I did two types of checks here, if Heureka.cz did so:

- I manually went through crawling log if there are long sequences of refused connections
- I manually visited exactly 200 random redirection links and did not found a link that was valid, but Heureka.cz refused the redirection to it

In [6]:
errors['true_domain'].value_counts()

Connection refused                          23238
404                                          1953
403                                           750
503                                           504
Got Exception named ReadTimeout               191
500                                           146
429                                            41
401                                            40
400                                            37
502                                            25
Got Exception named TooManyRedirects           20
410                                            18
406                                            17
520                                             9
521                                             8
402                                             6
504                                             6
523                                             4
301                                             4
530                                             3


## True Domains Validation
Obviously, from the list below, the dataset contains duplicates an not-anymore working websites.

In [7]:
tdf['true_domain'].value_counts()

https://www.shoptet.cz/smazan/                      223
http://www.eshop-zdarma.cz                          210
https://www.shoptet.cz/ukoncen-v-ochranne-lhute/    199
https://www.shoptet.cz/                              37
https://www.shoptet.cz/nespusten/                    32
                                                   ... 
https://www.ambere.cz/                                1
https://cupakdesign.com/produkty/                     1
https://www.carpstar.cz/                              1
http://www.silverbull.cz/                             1
https://ebezo.cz/                                     1
Name: true_domain, Length: 44593, dtype: int64

In [8]:
duplicates = tdf.groupby('true_domain').filter(lambda x: len(x) >= 2)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(duplicates['true_domain'].value_counts())

https://www.shoptet.cz/smazan/                                                                                                                    223
http://www.eshop-zdarma.cz                                                                                                                        210
https://www.shoptet.cz/ukoncen-v-ochranne-lhute/                                                                                                  199
https://www.shoptet.cz/                                                                                                                            37
https://www.shoptet.cz/nespusten/                                                                                                                  32
http://www.webuje.cz/404.asp                                                                                                                       23
http://tony.cz/                                                                                     

### Remove obviously not working sites

I manually went thought the list above and I created a list of suspicious websites that should be removed from the dataset. This was done iteratively, while I was continuously updating the remove list below.

In [9]:
remove_list = [ 
                "http://404.station.cz",
                "https://www.sluzby-zbozi.cz/obleceni-tehotenske/?_r=tehotenske-saty.cz",
                "http://www.velkoobchod-alukol.cz/redakce/rs/velkoobchod_alu_docasne/index.html",
                "https://www.isic.cz/",
                "https://napovime.cz/navod/jak-vybrat-brzdy-brembo/",
                "https://www.microsoft.com/cs-cz",
                "https://www.hugedomains.com/domain_profile.cfm?d=plysaci&e=com",
                "https://sedo.com/search/details/?partnerid=324561&language=cs&domain=hbshop.eu&origin=sales_lander_1&utm_medium=Parking&utm_campaign=offerpage",
                "https://www.ccshop.cz/?aid=2423",
                "https://www.shoptet.sk/zmazany/",
                "https://www.daukce.cz/domains/webyan.cz",
                "https://www.shoptet.sk/ukonceny-v-ochrannej-lehote/",
                "https://girlssexshow.com/live-sex-chats/college-cam-girls",
                "https://www.atomer.cz/zaporny-kredit/",
                "https://pipni.cz/403",
                "https://www.casinoarena.cz/rubriky/kasino-zajimavosti/neobux-recenze-a-strategie-diskuze-a-zkusenosti_10603.html",
                "https://www.webmium.cz/eshopy",
                "https://error.banan.cz/404-not-found.html",
                "https://www.wedos.cz/",
                "http://www.svet-stranek.cz/",
                "https://www.shopsun.cz/",
                "https://www.shoptet.cz/ukoncen/",
                "https://www.ftsun.cz/nase-produkty/vyvoj-aplikaci/krabicove-reseni.html?utm_source=eshop.evron.cz",
                "http://www.webuje.cz/404.asp",
                "https://www.shoptet.cz/nespusten/",
                "https://www.shoptet.cz/",
                "https://www.shoptet.cz/ukoncen-v-ochranne-lhute/",
                "http://www.eshop-zdarma.cz",
                "https://www.shoptet.cz/smazan/",
                "http://www.xxx.cz",
                "http://expirace.banan.cz",
                "http://expirace.banan.cz?blazasrot",
                "https://www.metrpiva.cz/chyba-404-nenalezeno.html",
                "https://www.domenyprodej.eu/domain/offer/404",
                "https://www.evans.cz/404",
                "https://www.belicipasky.cz/404.php",
                "https://www.podlahysevcik.cz/404.php?page=https://www.podlahysevcik.cz/404",
                "http://img.mimishop.cz/s/ms/1488/1/0128/b2840504.jpg"
               ]
tdf = tdf[~tdf['true_domain'].isin(remove_list)]

In [10]:
# Tried all of these and updated remove list
# banan.cz
# pipni.cz
# shoptet.cz
# 404
# 405
# 406
# 407
# 500
# wedos
# forpsi

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(tdf[tdf['true_domain'].str.contains('banan.cz')])

,name,heureka,true_domain


### Merge duplicates

In [44]:
# drops too many entries
# m1 = tdf['heureka'].isin(tdf.drop_duplicates('true_domain', keep=False)['heureka'])
# m2 = df.duplicated('heureka', keep=False)
# merged_tdf = tdf[m1 ^ m2]

In [45]:
merged_tdf = tdf.drop_duplicates('true_domain', keep='first')

### Find websites on the same domain

Manually filtered rows that has same domain name. Some of them are not valid eshops, or the websites do not work anymore.

I iteratively looked into the results of the commands bellow and I manually checked if the links are valid, if the eshops are actual eshops or if the webshop does not work anymore.

- eshop-rychle.cz - ok
- www.daukce.cz - delete, domain seller
- www.hugedomains.com - delete, domain seller
- www.sluzby-zbozi.cz - delete, always redirects to a page that is not a webshop
- www.kramky.cz - delete, it is a website that let a user to create a single page eshop
- www.hobbycentrum.cz - merge all together, it is a single website
- www.webareal.cz - ok
- www.mimishop.cz - delete, links are not valid anymore
- www.facebook.com - delete, it is not a webshop
- aukro.cz - delete, it is a czech ebay like site
- sedo.com - delete, domain seller
- www.fler.cz - replace with a single link
- www.amateri.com - it is not a webshop
- www.prodejonline.cz - remove, not working anymore 
- onlineshopy.com - remove, not working anymore
- www.webhouse.sk - remove, domain seller
- nova-domena-hosting-zdarma.cz - remove, domain seller
- nova-domena-hosting90.cz - remove, domain seller
- www.centrum-domen.eu - remove, domain seller
- labori.cz - remove, domain seller
- www.nonamedomain.hu - remove, domain seller
- www.domaincostclub.com - remove, domain seller

Rest of domains are merged to a single link.



In [46]:
from urllib.parse import urlparse
merged_tdf['true_domain_clean'] = merged_tdf['true_domain'].apply(lambda x: urlparse(x).netloc) 

<ipython-input-46-32fb9aeb13de>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_tdf['true_domain_clean'] = merged_tdf['true_domain'].apply(lambda x: urlparse(x).netloc)


In [47]:
duplicates = merged_tdf.groupby('true_domain_clean').filter(lambda x: len(x) >= 2)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(duplicates['true_domain_clean'].value_counts())
    

www.eshop-rychle.cz                153
www.daukce.cz                       74
www.hugedomains.com                 45
www.sluzby-zbozi.cz                 28
www.kramky.cz                       23
www.hobbycentrum.cz                 20
www.webareal.cz                     19
www.mimishop.cz                     17
www.facebook.com                    13
aukro.cz                            13
www.fler.cz                         12
www.amateri.com                     12
sedo.com                            12
www.prodejonline.cz                 10
www.altisport.cz                     9
www.grilykrby.cz                     9
www.alza.cz                          9
onlineshopy.com                      8
www.leano.cz                         8
www.truhlikov.cz                     8
www.svarecky-obchod.cz               8
www.modio.cz                         7
www.mall.cz                          7
www.mojegalanterka.cz                7
www.webhouse.sk                      7
www.mimibazar.cz         

In [48]:
# try one of the aboves
webpage = '__' 
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(merged_tdf[merged_tdf['true_domain_clean'] == webpage])

,name,heureka,true_domain,true_domain_clean


In [49]:
# try
# domen, domai and more
string = '__'
merged_tdf[merged_tdf['true_domain_clean'].str.contains(string)]

,name,heureka,true_domain,true_domain_clean


### Remove data with the respect to newly given information

In [50]:
remove_list = ['www.daukce.cz', 'www.hugedomains.com', 'www.sluzby-zbozi.cz',
               'www.kramky.cz', 'www.domaincostclub.com', 'www.mimishop.cz',
               'www.facebook.com', 'aukro.cz', 'sedo.com', 'www.amateri.com',
               'www.prodejonline.cz', 'onlineshopy.com', 'www.webhouse.sk',
               'nova-domena-hosting-zdarma.cz', 'nova-domena-hosting90.cz',
               'www.centrum-domen.eu', 'labori.cz', 'www.nonamedomain.hu']
cleaner_merged_tdf = merged_tdf[~merged_tdf['true_domain_clean'].isin(remove_list)]
cleaner_merged_tdf[cleaner_merged_tdf['true_domain_clean'] == 'www.webareal.cz']

,name,heureka,true_domain,true_domain_clean
627,e-cigarko,https://www.heureka.cz/exit/webareal-cz-e-ciga...,http://www.webareal.cz/e-cigarko,www.webareal.cz
1054,e-hemadobruska.cz,https://www.heureka.cz/exit/webareal-cz-tomsho...,http://www.webareal.cz/tomshop,www.webareal.cz
7790,DROGERIE - SKOBLA,https://www.heureka.cz/exit/webareal-cz-droger...,http://www.webareal.cz/drogerie-skobla,www.webareal.cz
10228,CLEAN POOL Čistý bazén,https://www.heureka.cz/exit/webareal-cz-cleanp...,http://www.webareal.cz/cleanpool,www.webareal.cz
14278,amulety-symboly,https://www.heureka.cz/exit/webareal-cz-amulet...,http://www.webareal.cz/amulety-symboly,www.webareal.cz
14305,DENORE,https://www.heureka.cz/exit/webareal-cz-denore...,http://www.webareal.cz/denore,www.webareal.cz
19037,Umikov CZ s.r.o.,https://www.heureka.cz/exit/webareal-cz-umikov...,http://www.webareal.cz/umikov,www.webareal.cz
23288,Petr Čížek,https://www.heureka.cz/exit/webareal-cz-cizek/...,http://www.webareal.cz/cizek,www.webareal.cz
23363,TEPLO DOMOVA Z BRIKET,https://www.heureka.cz/exit/webareal-cz-briket...,http://www.webareal.cz/brikety,www.webareal.cz
28135,Dřevěné hračky Kubů,https://www.heureka.cz/exit/webareal-cz-dreven...,http://www.webareal.cz/drevenehrackykubu,www.webareal.cz


In [51]:
# drops too many entries
# m1 = cleaner_merged_tdf['heureka'].isin(cleaner_merged_tdf.drop_duplicates('true_domain_clean', keep=False)['heureka'])
# m2 = cleaner_merged_tdf.duplicated('heureka', keep=False)
# merged_cleaner_merged_tdf = cleaner_merged_tdf[m1 ^ m2]

In [52]:
merged_cleaner_merged_tdf = cleaner_merged_tdf.drop_duplicates('true_domain_clean', keep='first')

In [53]:
merged_cleaner_merged_tdf['true_domain_clean'].value_counts()

www.ceskyraj.com          1
www.elegro.cz             1
www.petito.cz             1
www.nabytek-dreveny.cz    1
www.podlahy-vales.cz      1
                         ..
www.darka-shop.cz         1
www.game2k.cz             1
o3v.cz                    1
www.kasa.cz               1
www.ukaprika.cz           1
Name: true_domain_clean, Length: 43420, dtype: int64

In [54]:
final_df = merged_cleaner_merged_tdf.drop_duplicates(subset=['true_domain_clean'], keep='first')

## Last deletation of test names

In [55]:
final_df = final_df[final_df['name'] != 'test']

In [56]:
len(final_df)

43413

In [57]:
final_df[final_df['true_domain_clean'].str.contains('www.alza.cz')]

,name,heureka,true_domain,true_domain_clean
8648,vyberovavina.cz,https://www.heureka.cz/exit/vyberovavina-cz/?z=4,https://www.alza.cz/vina/18877547.htm?idp=1021...,www.alza.cz


## Saving the results


In [58]:
final_df.to_csv('clean_eshop_list.csv', index=False)